In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *

In [2]:
dataset = pd.read_csv("/Users/ryanlucas/Desktop/kc_house_data.csv")

In [3]:
dataset

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [4]:
train_idx = np.arange(0, int(0.7*len(dataset)))
test_idx = np.arange(int(0.7*len(dataset)), int(1*len(dataset)))

features = dataset.loc[train_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]

diagnosis = dataset.loc[train_idx, "price"]
orig_columns = features.copy().columns
orig_features_size = features.copy().shape[1]

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features = pd.DataFrame(sc.fit_transform(features))
features.columns = orig_columns

In [6]:
features_test = dataset.loc[test_idx, ["floors", "bedrooms",
    "bathrooms",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "sqft_living",
    "sqft_lot",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15"]]
diagnosis_test = dataset.loc[test_idx, 'price']

In [7]:
features_test = pd.DataFrame(sc.fit_transform(features_test))
features_test.columns = orig_columns

In [8]:
diagnosis

0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
15124    555700.0
15125    690000.0
15126    449000.0
15127    308500.0
15128    340000.0
Name: price, Length: 15129, dtype: float64

In [9]:
print(len(dataset))
print(len(features))
print(len(features_test))

21613
15129
6484


In [10]:
features

,floors,bedrooms,bathrooms,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living,sqft_lot,lat,long,sqft_living15,sqft_lot15
0,-0.847801,-0.396310,-1.396445,-0.703175,-0.674634,-0.436232,-0.217431,-0.961356,-0.239098,-0.348602,-0.301875,-0.940320,-0.278087
1,1.113119,-0.396310,0.251615,0.524394,0.213892,-0.579063,4.587486,0.573193,-0.201860,1.152626,-0.747374,-0.420453,-0.206627
2,-0.847801,-1.496047,-1.396445,-1.211562,-0.674634,-1.221803,-0.217431,-1.413993,-0.137349,1.273554,-0.129424,1.109440,-0.191429
3,-0.847801,0.703428,1.240451,-0.864371,1.346762,-0.079155,-0.217431,-0.100242,-0.254302,-0.279909,-1.279099,-0.910614,-0.301440
4,-0.847801,-0.396310,-0.077997,-0.083190,-0.674634,0.706416,-0.217431,-0.409360,-0.182259,0.407021,1.221446,-0.257067,-0.211513
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,-0.847801,-0.396310,-0.077997,0.078006,-0.674634,-0.364817,-0.217431,-0.265841,-0.080744,0.206666,0.632237,0.381627,-0.038090
15125,1.113119,0.703428,0.581227,1.826362,-0.674634,0.849247,-0.217431,1.290787,0.456770,1.393767,0.725648,1.599600,0.785444
15126,-0.847801,-0.396310,-0.407609,-0.107990,-0.674634,-0.543356,-0.217431,-0.431440,-0.144436,1.405931,-0.510254,0.129120,0.259894
15127,-0.847801,-1.496047,-1.396445,-1.112364,-0.674634,-0.614771,-0.217431,-1.325673,-0.226841,1.254234,-0.812044,-1.296800,-0.259261


In [ ]:
np.random.seed(1)
n = 5
rules = {}
for i in range(1,20):

    #################
    sampled_features = features.copy().sample(n=int(n),axis='columns')

    OCT_H = iai.GridSearch(
          iai.OptimalTreeRegressor(
              max_depth = 3, cp = 0.00000000001))

    OCT_H.fit(sampled_features, diagnosis)
    OCT_H.write_json(f"tre_{i}.json")

    ######################
    f = open(f"tre_{i}.json")
    data = json.load(f)

    paths = gen_paths(data)

    sub_paths = gen_subpaths(paths)

    names = sampled_features.columns

    rule_features_new = gen_features(
            paths, data, sampled_features, names)

    rules[f"tre_{i}"] = rule_features_new

    print(f"Tree: {i}")


In [ ]:
features.columns

In [ ]:
big_df = pd.DataFrame()
for tree in rules.keys():

    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [ ]:
df = big_df.loc[:,~big_df.columns.duplicated()].copy()

In [ ]:
for column in df.columns[1:]:
    if df[column].sum() == len(df):
        del df[column]


In [ ]:
df

In [ ]:
features_for_robust_reg = df

In [ ]:
rule_features = features_for_robust_reg.copy()
features_for_robust_reg = pd.concat([features, features_for_robust_reg], axis = 1)

In [ ]:
features_for_robust_reg.sum()

In [ ]:
features_for_robust_reg.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_for_robust_reg))])

In [ ]:
features_for_robust_reg

In [ ]:
features_for_robust_reg

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
features.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features))])

In [ ]:
reg_baseline = LinearRegression().fit(features.copy()
, diagnosis)

In [ ]:
θ_baseline = reg_baseline.coef_

In [ ]:
predictions_baseline = reg_baseline.predict(features.copy())

In [ ]:
(diagnosis - predictions_baseline).describe()

In [ ]:
SSE_model = sum((diagnosis - predictions_baseline)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
features_test.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_test))])

In [ ]:
predictions_test_baseline = reg_baseline.predict(features_test)

In [ ]:
SSE_model = sum((diagnosis_test - predictions_test_baseline)**2)
SSE_mean = sum((diagnosis_test - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
reg_trees = LinearRegression().fit(features.copy()
, diagnosis)

In [ ]:
from sklearn import linear_model

In [ ]:
lasso_trees = linear_model.Ridge(alpha = 1).fit(features_for_robust_reg.copy()
, diagnosis)

In [ ]:
predictions_lasso_trees = lasso_trees.predict(features_for_robust_reg)

In [ ]:
SSE_model = sum((predictions_lasso_trees - diagnosis)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
features = features_test
for rule in rule_features.columns:

    index_feature = list(features.loc[eval(rule)].index)

    print(index_feature)
    new_feature = [1 if i in index_feature else 0 for i in range(features.index[0], features.index[-1]+1)]

    features[rule] = new_feature




In [ ]:
new_features = features_test

In [ ]:
new_features.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(new_features))])

In [ ]:
predictions_test = lasso_trees.predict(features_test)

In [ ]:
plt.scatter(predictions_test, diagnosis_test)

In [ ]:
SSE_model = sum((diagnosis_test - predictions_test)**2)
SSE_mean = sum((diagnosis_test - diagnosis.mean())**2)

1 - SSE_model/SSE_mean